Cryptocompare  
https://developers.cryptocompare.com/documentation/data-api/spot_v1_historical_hours
biance,bybit,okx,bitget,coinbase,kraken,upbit,gateio,bitfinex,gemini,binanceusa,bingx,bitmex,bitstamp,coinbaseinternational,kucoin,

In [ ]:
import os
import time
import requests
import pandas as pd
from datetime import datetime, timedelta

API_KEY = "2597f84529bce37c53b227e21ba471abb6d58a6e5090ed4f3993be77313729e4" # cryptocompare api key 超过了就再注册一个

In [17]:
url_markets = "https://data-api.cryptocompare.com/spot/v1/markets"

headers = {"Content-type": "application/json", "authorization": f"Apikey {API_KEY}"}
response = requests.get(url_markets, headers=headers)

if response.status_code == 200:
    data = response.json()["Data"]

    # **提取所有 market（交易所）**
    markets = [{"market": market, **details} for market, details in data.items()]
    markets_df = pd.DataFrame(markets)
    print(f"✅ 获取 {len(markets_df)} 个交易所")

else:
    print(f"❌ API 请求失败: {response.status_code}, {response.text}")

markets_df.head()

✅ 获取 176 个交易所


,market,TYPE,EXCHANGE_STATUS,MAPPED_INSTRUMENTS_TOTAL,UNMAPPED_INSTRUMENTS_TOTAL,INSTRUMENT_STATUS,TOTAL_TRADES_SPOT,HAS_ORDERBOOK_L2_MINUTE_SNAPSHOTS_ENABLED
0,aax,602,ACTIVE,250,185,"{'ACTIVE': 88, 'IGNORED': 0, 'RETIRED': 347, '...",648891006,False
1,abcc,602,ACTIVE,304,28,"{'ACTIVE': 29, 'IGNORED': 0, 'RETIRED': 303, '...",8459290,False
2,acx,602,ACTIVE,0,32,"{'ACTIVE': 32, 'IGNORED': 0, 'RETIRED': 0, 'EX...",6203612,False
3,aidosmarket,602,ACTIVE,1,2,"{'ACTIVE': 2, 'IGNORED': 0, 'RETIRED': 1, 'EXP...",155608,False
4,alphaex,602,ACTIVE,10,2,"{'ACTIVE': 2, 'IGNORED': 0, 'RETIRED': 10, 'EX...",1718824,False


In [ ]:
selected_markets = [
    "binance", "binanceaggregate", "binanceusa", "bingx", "bitfinex",
    "bitget", "bitmex", "bitstamp", "bybit", "coinbase",
    "coinbaseinternational", "gateio", "gemini", "kraken", "kucoin",
    "okex", "upbit"
]


In [22]:
url_instruments = "https://data-api.cryptocompare.com/spot/v1/markets/instruments"
headers = {"Content-type": "application/json", "authorization": f"Apikey {API_KEY}"}

all_instruments = []

for market in selected_markets:
    params = {"market": market}
    response = requests.get(url_instruments, headers=headers, params=params)

    json_response = response.json()
    market_data = json_response["Data"].get(market, {})
    instruments_data = market_data.get("instruments", {})

    # 遍历market的所有交易对
    for instrument_name, instrument_details in instruments_data.items():
        all_instruments.append({
            "market": market,
            "instrument": instrument_name,
            "base_asset": instrument_details.get("INSTRUMENT", "N/A"),
            "status": instrument_details.get("INSTRUMENT_STATUS", "UNKNOWN")
        })

    print(f"✅ {market} 交易所的 {len(instruments_data)} 个交易对已获取")

    # 避免过快请求
    time.sleep(0.5)

instruments_df = pd.DataFrame(all_instruments)
instruments_df.head()

save_dir = "C:/Users/YuweiCao/Documents/GitHub/Project/Project/spot_volume/result"
csv_filepath = os.path.join(save_dir, "crypto_instruments.csv")
instruments_df.to_csv(csv_filepath, index=False, encoding="utf-8")


✅ okex 交易所的 694 个交易对已获取


In [ ]:
instrument_filepath = "C:/Users/YuweiCao/Documents/GitHub/Project/Project/spot_volume/result/crypto_instruments.csv"
instruments_df = pd.read_csv(instrument_filepath)

filtered_instruments_df = instruments_df[
    instruments_df["instrument"].str.endswith(("-USDT", "-USD", "-USDC", "-KRW"))
]

print(f"✅ 筛选后剩余 {len(filtered_instruments_df)} 个交易对")

# **CryptoCompare API 端点**
url_spot_volume = 'https://data-api.cryptocompare.com/spot/v1/historical/hours'
headers = {"Content-type": "application/json", "authorization": f"Apikey {API_KEY}"}

# **存储路径**
save_dir = "C:/Users/YuweiCao/Documents/GitHub/Project/Project/spot_volume/result"
os.makedirs(save_dir, exist_ok=True)  # 确保存储文件夹存在

# **遍历所有交易所**
for market in filtered_instruments_df["market"].unique():
    trade_data = []  # **存储数据**

    # **筛选当前交易所的交易对**
    market_instruments_df = filtered_instruments_df[filtered_instruments_df["market"] == market]

    for _, row in market_instruments_df.iterrows():
        instrument = row["instrument"]

        params = {
            "market": market,
            "instrument": instrument,
            "limit": 2000,  # 获取最近 2000 小时数据
            "aggregate": 1,  # 每一个小时
            "fill": "true",
            "apply_mapping": "true",
            "response_format": "JSON",
        }

        response = requests.get(url_spot_volume, headers=headers, params=params)

        if response.status_code == 200:
            json_response = response.json()

            # **检查数据是否有效**
            if "Data" in json_response and isinstance(json_response["Data"], list) and len(json_response["Data"]) > 0:
                for entry in json_response["Data"]:  # 遍历 2000 小时数据
                    trade_data.append({
                        "market": market,
                        "instrument": instrument,
                        "base_asset": entry["BASE"],  # 基础资产
                        "quote_asset": entry["QUOTE"],  # 报价资产
                        "time": pd.to_datetime(entry["TIMESTAMP"], unit="s"),  # 转换时间
                        "open": entry["OPEN"],  # 开盘价
                        "high": entry["HIGH"],  # 最高价
                        "low": entry["LOW"],  # 最低价
                        "close": entry["CLOSE"],  # 收盘价
                        "total_trades": entry["TOTAL_TRADES"],  # 总交易笔数
                        "total_trades_buy": entry["TOTAL_TRADES_BUY"],  # 买单数
                        "total_trades_sell": entry["TOTAL_TRADES_SELL"],  # 卖单数
                        "volume": entry["VOLUME"],  # 以 base_asset 计数的交易量
                        "quote_volume": entry["QUOTE_VOLUME"],  # 以 quote_asset 计数的交易量
                        "volume_buy": entry["VOLUME_BUY"],  # 以 base_asset 计数的买单交易量
                        "quote_volume_buy": entry["QUOTE_VOLUME_BUY"],  # 以 quote_asset 计数的买单交易量
                        "volume_sell": entry["VOLUME_SELL"],  # 以 base_asset 计数的卖单交易量
                        "quote_volume_sell": entry["QUOTE_VOLUME_SELL"],  # 以 quote_asset 计数的卖单交易量
                    })
                print(f"✅ Data successfully fetched: {market} - {instrument} ({len(json_response['Data'])} hours)")
            else:
                print(f"❌ No data for {market} - {instrument}")

        else:
            print(f"❌ API request failed for {market} - {instrument}, Status Code: {response.status_code}")

        time.sleep(0.5)  # **避免 API 超时**

    # **转换成 DataFrame**
    if trade_data:
        spot_volume_df = pd.DataFrame(trade_data)

        # **存储 CSV，按交易所划分**
        csv_filepath = os.path.join(save_dir, f"{market}_spot_volume_2000h.csv")
        spot_volume_df.to_csv(csv_filepath, index=False, encoding="utf-8")
        print(f"📁 Data for {market} saved to {csv_filepath}")

✅ 筛选后剩余 593 个交易对
✅ Data successfully fetched: okex - GALA-USDT (2000 hours)
✅ Data successfully fetched: okex - BABYDOGE-USDT (2000 hours)
✅ Data successfully fetched: okex - JST-USDT (2000 hours)
✅ Data successfully fetched: okex - KDA-USDT (2000 hours)
✅ Data successfully fetched: okex - DGB-USDT (2000 hours)
✅ Data successfully fetched: okex - AERGO-USDT (2000 hours)
✅ Data successfully fetched: okex - LAT-USDT (2000 hours)
✅ Data successfully fetched: okex - ETC-USDT (2000 hours)
✅ Data successfully fetched: okex - DEP-USDT (2000 hours)
✅ Data successfully fetched: okex - API3-USDT (2000 hours)
✅ Data successfully fetched: okex - ALGO-USDT (2000 hours)
✅ Data successfully fetched: okex - AUCTION-USDT (2000 hours)
✅ Data successfully fetched: okex - ILV-USDT (2000 hours)
✅ Data successfully fetched: okex - CELO-USDT (2000 hours)
✅ Data successfully fetched: okex - BTC-USDT (2000 hours)
✅ Data successfully fetched: okex - DOT-USDT (2000 hours)
✅ Data successfully fetched: okex - ELON

In [ ]:
# **选择一个交易所 & 交易对**
market = "binance"
instrument = "BTC-USDT"

# **CryptoCompare API 端点**
url_spot_volume = 'https://data-api.cryptocompare.com/spot/v1/historical/hours'
headers = {"Content-type": "application/json", "authorization": f"Apikey {API_KEY}"}

params = {
    "market": market,
    "instrument": instrument,
    "limit": 2000,  # 获取最近 2000 小时数据
    "aggregate": 1,  # 每一个小时
    "fill": "true",
    "apply_mapping": "true",
    "response_format": "JSON",
}

response = requests.get(url_spot_volume, headers=headers, params=params)

if response.status_code == 200:
    json_response = response.json()

    # **检查数据是否有效**
    if "Data" in json_response and isinstance(json_response["Data"], list) and len(json_response["Data"]) > 0:
        trade_data = []

        for entry in json_response["Data"]:  # 遍历 2000 小时数据
            trade_data.append({
                "market": market,
                "instrument": instrument,
                "base_asset": entry["BASE"],  # 基础资产
                "quote_asset": entry["QUOTE"],  # 报价资产
                "time": pd.to_datetime(entry["TIMESTAMP"], unit="s"),  # 转换时间
                "open": entry["OPEN"],  # 开盘价
                "high": entry["HIGH"],  # 最高价
                "low": entry["LOW"],  # 最低价
                "close": entry["CLOSE"],  # 收盘价
                "total_trades": entry["TOTAL_TRADES"],  # 总交易笔数
                "total_trades_buy": entry["TOTAL_TRADES_BUY"],  # 买单数
                "total_trades_sell": entry["TOTAL_TRADES_SELL"],  # 卖单数
                "volume": entry["VOLUME"],  # 以 base_asset 计数的交易量
                "quote_volume": entry["QUOTE_VOLUME"],  # 以 quote_asset 计数的交易量
                "volume_buy": entry["VOLUME_BUY"],  # 以 base_asset 计数的买单交易量
                "quote_volume_buy": entry["QUOTE_VOLUME_BUY"],  # 以 quote_asset 计数的买单交易量
                "volume_sell": entry["VOLUME_SELL"],  # 以 base_asset 计数的卖单交易量
                "quote_volume_sell": entry["QUOTE_VOLUME_SELL"],  # 以 quote_asset 计数的卖单交易量
            })

        print(f"✅ Data successfully fetched: {market} - {instrument} ({len(json_response['Data'])} hours)")

        # **转换成 DataFrame**
        trade_volume_df = pd.DataFrame(trade_data)

        # **存储 CSV**
        save_dir = "C:/Users/YuweiCao/Documents/GitHub/Project/Project/spot_volume/result"
        os.makedirs(save_dir, exist_ok=True)  # 确保文件夹存在
        csv_filepath = os.path.join(save_dir, f"{market}_{instrument}_spot_volume_2000h.csv")
        trade_volume_df.to_csv(csv_filepath, index=False, encoding="utf-8")

        # **显示前 10 行的数据**
    else:
        print(f"❌ No data for {market} - {instrument}")

else:
    print(f"❌ API request failed for {market} - {instrument}, Status Code: {response.status_code}")


✅ Data successfully fetched: binance - BTC-USDT (2000 hours)


In [31]:
import os
import pandas as pd

# **数据存储目录**
data_dir = "C:/Users/YuweiCao/Documents/spot_volume_result"

# **获取所有 CSV 文件**
csv_files = [f for f in os.listdir(data_dir) if f.endswith(".csv")]

# **存储异常交易量的交易对**
anomalies = []

# **遍历 CSV 文件**
for file in csv_files:
    file_path = os.path.join(data_dir, file)
    
    try:
        # **读取数据**
        df = pd.read_csv(file_path)

        # **检查 volume 是否异常**
        zero_volume = df[df["volume"] == 0]
        negative_volume = df[df["volume"] < 0]
        extreme_volume = df[df["volume"] > df["volume"].quantile(0.999)]  # 99.9% 分位数之外的数据
        
        # **存储异常数据**
        if not zero_volume.empty:
            anomalies.append((file, "Zero Volume", len(zero_volume)))
        if not negative_volume.empty:
            anomalies.append((file, "Negative Volume", len(negative_volume)))
        if not extreme_volume.empty:
            anomalies.append((file, "Extreme Volume", len(extreme_volume)))

        print(f"✅ 交易量检查完成: {file}")

    except Exception as e:
        print(f"❌ 处理 {file} 时出错: {e}")

# **输出异常情况**
if anomalies:
    print("\n🚨 发现异常交易量:")
    for anomaly in anomalies:
        print(f"❌ 文件: {anomaly[0]}, 异常类型: {anomaly[1]}, 记录数: {anomaly[2]}")
else:
    print("\n✅ 所有交易量数据正常！")



✅ 交易量检查完成: binanceaggregate_spot_volume_2000h.csv
✅ 交易量检查完成: binanceusa_spot_volume_2000h.csv
✅ 交易量检查完成: binance_spot_volume_2000h.csv
✅ 交易量检查完成: bingx_spot_volume_2000h.csv
✅ 交易量检查完成: bitfinex_spot_volume_2000h.csv
✅ 交易量检查完成: bitget_spot_volume_2000h.csv
✅ 交易量检查完成: bitmex_spot_volume_2000h.csv
✅ 交易量检查完成: bitstamp_spot_volume_2000h.csv
✅ 交易量检查完成: bybit_spot_volume_2000h.csv
✅ 交易量检查完成: coinbaseinternational_spot_volume_2000h.csv
✅ 交易量检查完成: coinbase_spot_volume_2000h.csv
❌ 处理 crypto_instruments.csv 时出错: 'volume'
✅ 交易量检查完成: gateio_spot_volume_2000h.csv
✅ 交易量检查完成: gemini_spot_volume_2000h.csv
✅ 交易量检查完成: kraken_spot_volume_2000h.csv
✅ 交易量检查完成: kucoin_spot_volume_2000h.csv
✅ 交易量检查完成: okex_spot_volume_2000h.csv
✅ 交易量检查完成: upbit_spot_volume_2000h.csv

🚨 发现异常交易量:
❌ 文件: binanceaggregate_spot_volume_2000h.csv, 异常类型: Zero Volume, 记录数: 1560
❌ 文件: binanceaggregate_spot_volume_2000h.csv, 异常类型: Extreme Volume, 记录数: 1003
❌ 文件: binanceusa_spot_volume_2000h.csv, 异常类型: Zero Volume, 记录数: 134716
❌ 文件: binan